In [11]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder
import joblib

## Droping cols and OneHotEncoding aka set dummy variable

In [12]:
def preprocess(csv_file, num_of_rows):
  csv_file = csv_file.drop([
    'Unnamed: 0'
    #,'month', 
    #'flat_type', 
    #'address',
    #'lease_commence_date'
    # these 5, try removing to see the effects on data
    #,'year'
    #,'years'
    #,'quarter'
    #,'Lat'
    #,'Lon'
    ],axis = 1)

  onehot = OneHotEncoder()

  town_df = pd.DataFrame(onehot.fit_transform(csv_file[['town']]).toarray())
  town_df.columns = onehot.get_feature_names_out(['town'])

  flat_model_df = pd.DataFrame(onehot.fit_transform(csv_file[['flat_model']]).toarray())
  flat_model_df.columns = onehot.get_feature_names_out(['flat_model'])

  csv_file = csv_file.join([town_df, flat_model_df])
  csv_file = csv_file.drop(['town','flat_model'], axis = 1)


  add_flat = np.zeros(num_of_rows)
  ## since flat_model_Adjoined flat is not in the data 
  csv_file["flat_model_Adjoined flat"] = add_flat.copy()
  csv_file["flat_model_2-room"] = add_flat.copy()
  csv_file["flat_model_Model A-Maisonette"] = add_flat.copy()
  csv_file["flat_model_Multi Generation"] = add_flat.copy()
  csv_file["flat_model_Premium Maisonette"] = add_flat.copy()
  #train = pd.read_csv("../data/df2017.csv")
  #train = train.drop(['Unnamed: 0', 'month', 'town', 'flat_type', 'flat_model', 'lease_commence_date', 'year', 'quarter', 'address', 'years', 'k_fold'],axis = 1)
  cols = ['floor_area_sqm', 'resale_price', 'remaining_lease', 'index', 'Lat', 'Lon', 'dist_presch', 'dist_gym', 'dist_park', 'dist_pharm', 
    'dist_mrt', 'dist_hawker', 'storey_index', 'town_BUKIT BATOK', 'town_BUKIT MERAH', 'town_CLEMENTI', 'town_JURONG WEST', 
    'town_SENGKANG', 'town_TAMPINES', 'town_TOA PAYOH', 'town_WOODLANDS', 'flat_model_2-room', 'flat_model_Apartment', 'flat_model_DBSS', 
    'flat_model_Improved', 'flat_model_Maisonette', 'flat_model_Model A', 'flat_model_Model A-Maisonette', 'flat_model_Model A2', 
    'flat_model_Multi Generation', 'flat_model_New Generation', 'flat_model_Premium Apartment', 'flat_model_Premium Maisonette', 
    'flat_model_Simplified', 'flat_model_Standard', 'flat_model_Adjoined flat']
  #print(cols)
  ## reorder back to the original col order
  csv_file = csv_file[cols]

  return csv_file


In [13]:
def test(testset_path):
  testset = pd.read_csv(testset_path)
  #savefile = open("savefile.txt", "w")
  model = joblib.load("savedModel2.joblib")

  num_of_rows = len(testset.index)
  testset2 = preprocess(testset, num_of_rows)

  y_test = testset2['resale_price'].tolist()
  x_test = testset2.drop(['resale_price'], axis = 1).to_numpy()

  y_test_pred = model.predict(x_test)

  percentage_error = (y_test_pred - y_test) / y_test *100
  testset["pred"] = y_test_pred
  testset["perc_error"] = percentage_error
  testset.to_csv("alltownpred.csv")
  #savefile.write(f'{percentage_error}')
  #savefile.flush()
  print(sum(abs(percentage_error)) / num_of_rows)
  return percentage_error
    

In [14]:
pd.read_csv("../data/s2sometowns.csv").head()

,Unnamed: 0,town,flat_type,floor_area_sqm,flat_model,remaining_lease,resale_price,address,ind,Lat,Lon,dist_presch,dist_gym,dist_park,dist_pharm,dist_mrt,dist_hawker,index,storey_index
0,1,BUKIT BATOK,3 ROOM,67,New Generation,62,360000.0,239 Bt Batok East Ave 5,104,1.350307,103.755124,91.885656,877.709784,792.263766,404.766178,579.100773,1577.521497,164.149516,4
1,2,BUKIT BATOK,3 ROOM,67,New Generation,61,330000.0,215 Bt Batok St 21,105,1.346801,103.753494,17.796610,1237.481267,1132.645899,393.380967,489.647770,1601.075717,164.149516,1
2,3,BUKIT BATOK,3 ROOM,68,New Generation,62,348000.0,305 Bt Batok St 31,106,1.357860,103.747968,292.947433,562.647516,903.344721,650.857550,439.246450,744.313004,164.149516,3
3,4,BUKIT BATOK,3 ROOM,68,New Generation,64,372000.0,363 Bt Batok St 31,107,1.360958,103.749237,182.159632,541.276120,859.447594,1007.890465,348.130945,1043.546274,164.149516,2
4,5,BUKIT BATOK,3 ROOM,68,New Generation,62,350000.0,312 Bt Batok St 32,108,1.359924,103.747539,126.263572,650.323814,991.294584,880.947573,475.693514,824.089999,164.149516,4


In [15]:
test("../data/s2alltowns.csv")

8.797588766975853


array([-16.09427954,  -6.75986283,  -6.19530388, ...,   1.76174781,
        -8.46561817,  -3.433839  ])

In [10]:
test("../data/s2sometowns.csv")

7.23123860426131


array([-6.09487957e+00, -1.72345269e+00, -4.86323964e+00, -1.01569563e+01,
       -3.98843816e+00,  3.29611502e+00,  4.46785398e-02,  9.45557634e+00,
        6.87757868e+00, -2.74398720e+00, -4.57768944e+00,  7.11271783e+00,
       -2.50769455e+00, -1.85415369e+01, -1.65944282e+01, -1.18904795e+01,
        4.56757469e-01, -2.26682496e+00, -2.92561795e+00,  4.73951764e+00,
       -7.99664104e+00, -2.92573108e+00, -7.88306760e+00,  2.82183497e+00,
       -1.17560782e+00, -1.12427747e+01, -5.47494357e+00, -8.89365638e+00,
       -1.35850728e+01, -1.33825819e+01, -5.00384746e+00,  5.00781602e+00,
       -1.15758059e+01,  1.30179727e+01, -3.35619178e+01, -5.46870977e+00,
        8.65699395e-01,  1.22152245e+01, -1.58317351e+01, -1.10900172e+01,
        3.13212121e+00, -6.18646208e+00,  1.17661414e+01, -1.16214278e+01,
        2.16525444e+00,  7.22751511e+00,  2.92124914e+00, -5.21359286e+00,
       -1.09749663e+01, -9.43191507e+00, -6.98770468e+00, -1.06613181e+01,
        8.79187997e+00,  